In [1]:
# BERT 학습을 위한 전처리
# BERT의 두 가지 언어 모델 학습에 대응하는 것으로 이를 표현하기 위해 세 개의 임베딩을 이용한다.
# 그 중 첫째는 토큰 임베딩이다. 문장에 사용된 단어 외에 두 종류의 특수 토큰이 추가돼 있다. 
# 첫째는 [CLS]토큰: 분류 토큰으로, 한 문서에 대한 문서 분류나 두 문서의 관계에 대한 분류를 하기 위한 정보를 수집해 최종적으로 출력 
# 둘째는 [SEP]토큰: seperator 토큰으로 한 문장의 끝을 나타내거나 두 문장을 분리한다.

# 둘째 임베딩은 구간 임베딩이며, 문장을 구분한다.[CLS]와 첫 문장의 토큰 그리고 첫 문장의 끝 [SEP]까지를 보통0, 나머지를 1로 임베딩
# 셋째 임베딩은 위치 임베딩으로 시퀀스에서의 순서를 나타낸다. 셀프 어텐션을 사용하므로 RNN과 달리 각 토큰의 위치에 대한 정보는 없다.
# 따라서 이를 임베딩에 명시적으로 넣어준다. 따라서 일반적으로 BERT 토크나이저가 위치 임베딩을 반환하지는 않는다.

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

sentence1 = 'What a beautiful day!'
sentence2 = 'Nvidia Titan XP has 12GB of VRAM'

# 1. 토큰화 결과
print(sentence1, '토큰화 결과:', tokenizer.tokenize(sentence1))
print(sentence2, '토큰화 결과:', tokenizer.tokenize(sentence2))

What a beautiful day! 토큰화 결과: ['what', 'a', 'beautiful', 'day', '!']
Nvidia Titan XP has 12GB of VRAM 토큰화 결과: ['n', '##vid', '##ia', 'titan', 'xp', 'has', '12', '##gb', 'of', 'vr', '##am']


In [3]:
# input_ids 토큰 임베딩의 결과
# token_type_ids 구간 임베딩의 결과
# attention_mask 임베딩이 아닌 마스킹과 관련된 부분으로, 0이면 마스킹이 되어 셀프 어텐션에서 제외되고, 1이면 셀프 어텐션 포함
# 즉 입력 토큰에서 셀프 어텐션이 필요 없는 부분들은 마스킹을 한다.

# 2. BERT 모형 입력 생성
inputs = tokenizer([sentence1, sentence2], padding=True)
print('BERT 입력:', inputs)

BERT 입력: {'input_ids': [[101, 2054, 1037, 3376, 2154, 999, 102, 0, 0, 0, 0, 0, 0], [101, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [4]:
# 101토큰은 [CLS] , 102토큰은 [SEP]

# 3. 두 문장으로 이루어진 시퀀스에 대한 BERT 모형 입력 생성
inputs = tokenizer(sentence1, sentence2, padding=True)
print('두 문장 시퀀스에 대한 BERT 입력:', inputs)

두 문장 시퀀스에 대한 BERT 입력: {'input_ids': [101, 2054, 1037, 3376, 2154, 999, 102, 1050, 17258, 2401, 16537, 26726, 2038, 2260, 18259, 1997, 27830, 3286, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [5]:
# 트랜스포머의 트레이너를 이용한 미세조정학습
# 매우 복잡한 과정을 거쳐야 하지만 세밀하게 모형을 조정하는 것이 가능하고 자신이 원하는 다양한 딥러닝 모델을 추가해 모형을 확장하는 가능
# NLTK 영화리뷰 데이터 미세조정학습
import nltk
from nltk.corpus import movie_reviews

# sklearn에서 제공하는 split 함수를 사용
from sklearn.model_selection import train_test_split
import numpy as np

nltk.download('movie_reviews')
fileids = movie_reviews.fileids() # movie_reviews data에서 file id를 가져옴

# file id를 이용해 raw text file을 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids]
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids]

# label을 0,1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = [label_dict[c] for c in categories]

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)
print('Train set count:',len(X_train))
print('Test set count:', len(X_test))

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Leedonghoon\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Train set count: 1600
Test set count: 400


In [6]:
from transformers import BertTokenizerFast, BertForSequenceClassification

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

train_input = tokenizer(X_train, truncation=True, padding=True, return_tensors = 'pt')
test_input = tokenizer(X_test, truncation=True, padding=True, return_tensors='pt')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import torch

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key,val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    
    def __len__(self):
        return len(self.labels)
    
train_dataset = OurDataset(train_input, y_train)
test_dataset = OurDataset(test_input, y_test)

In [8]:
# 학습을 수행하기 전에 정확도 측정을 위한 준비를 한다. 트랜스포머가 제공하는 Trainer 클래스는 기본적으로 학습 도중에 손실에 대한 값만 제공
from datasets import load_metric

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions = predictions, references=labels)

C:\Users\Leedonghoon\AppData\Local\Temp\ipykernel_1372\3497906201.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [ ]:
# Trainer와 TrainingArguments 정의
# TrainingArguments는 학습에 사용할 다양한 옵션과 하이퍼 파라미터를 정의할 수 있게 지원하는 클래스이다.
# 반드시 필요한 네 개의 매개변수만 정의한다.
# 첫째, 체크포인트 모형을 저장할 폴더 위치, 체크포인트 모형은 모형이 가진 변수, 즉 학습된 결과
# 둘째, 학습 에포크로 전체 데이터셋에 대한 학습 횟수
# 셋째, per_device_train_batch_size는 학습에서 사용할 데이터 배치의 크기

from transformers import Trainer, TrainingArguments

# Trainer에서 사용할 하이퍼 파라미터 지정
training_args = TrainingArguments(
    output_dir = '/results', # 모형 예측이나 체크포인트 출력 폴더, 반드시 필요함
    num_train_epochs = 2,   # 학습 에포크 수
    per_device_train_batch_size = 8, # 학습에 사용할 배치 사이즈
    per_device_eval_batch_size = 16, # 평가에 사용할 배치 사이즈
)

trainer = Trainer(
    model=model,       # 학습할 모형
    args = training_args,     # 위에서 정의한 학습 매개변수
    train_dataset = train_dataset,   # 학습 데이터셋
    compute_metrics = compute_metrics,
)

# 미세조정학습 실행
trainer.train()

C:\Users\Leedonghoon\anaconda3\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\Leedonghoon\AppData\Local\Temp\ipykernel_1372\994940297.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key,val in self.inputs.items()}


Step,Training Loss


In [ ]:
# 89%로 지금까지보다 확실히 개선된 성능을 보여준다. 이 결과는 학습할 때 랜덤하게 초기화되거나 변경되는 변수들이 있어 항상 같지는 않다.
trainer.evaluate(eval_dataset=test_dataset)

In [ ]:
# 파이토치를 이용한 미세조정학습
# 트랜스포머의 Trainer를 사용하지 않고 파이토치로 미세조정학습을 하는 방법을 알아본다.
# Trainer는 비교적 간편하게 사전학습 모형에 대한 미세조정학습을 수행할 수 있다는 장점이 있는 반면,
# 모형을 필요에 따라 변경할 수 없다는 제한이 있다.
# 만일 BERT의 기본 모형을 사용하고 BERT가 제공하는 [CLS] 토큰의 출력값이나, 모든 단어들에 대한 출력값을 직접 사용하고 싶다면 파이토치를
# 이용해서 직접 모형을 수정하는 방법을 사용해야 한다.
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=8)

In [ ]:
# 분류기가 없는 원형의 BERT 모형에 직접 분류기를 추가해 감성 분석을 할 수 있는 사용자정의 모형을 만들고 그 모형을 학습
# 언어 모델 사전학습 모형인 BertModel 사용
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
# BERT 사전학습 모형을 포함하는 신경망 모형을 선언
# nun_labels 분류할 클래스의 수, 감성분석 2
# token_size BERT 모형의 출력 벡터의 크기, 문서 분류에서는 모든 단어들의 출략(임베딩) 벡터가 아니라 CLS 토큰에 대응하는 출력 벡터만
# 가장 앞에 있는 CSL토큰만 outputs.last_hidden_state[:,0,:]
# 만약 양방향 LSTM이나 CNN 모형을 넣고 싶다면 추가하면 된다.

# BERT를 포함한 신경망 모형
class MyModel(torch.nn.Module):
    def __init__(self, pretrained_model, token_size, num_labels):
        super(MyModel, self).__init__()
        self.token_size = token_size
        self.num_labels = num_labels
        self.pretrained_model = pretrained_model
        # 분류기 정의
        self.classifier = torch.nn.Linear(self.token_size, self.num_labels)
        
    def forward(self, inputs):
        # BERT 모형에 입력을 넣고 출력을 받음
        outputs = self.pretrained_model(**inputs)
        # BERT 출력에서 CLS 토큰에 해당하는 부분만 가져옴
        bert_clf_token = outputs.last_hidden_state[:,0,:]
        
        return self.classifier(bert_clf_token)
    
# token_size는 BERT 토큰과 동일, bert_model.config.hidden_size로 알 수 있음
model = MyModel(bert_model, num_labels = 2, token_size=bert_model.config.hidden_size)

In [ ]:
# 학습
# 학습과정을 이해하려면 딥러닝의 학습 원리 및 단계와 파이토치에 대해 알아야 한다.
# 먼저 GPU 가속을 활성화하고 모형을 GPU로 복사한다. 다음은 학습을 위해 옵티마이저와 손실 함수를 정의한다.
# 각 에포크에서는 그레디언트를 초기화하고 모형으로 배치의 입력값에 대해 예측을 한 후, 답(labels)과 비교해 손실을 계산한다.
# 이에 따라 그래디언트를 계산하고 모형의 가중치를 수정한다.
from transformers import AdamW
import torch.nn.functional as F
import time

# GPU 가속을 사용할 수 있으면 device를 cuda로 설정하고, 아니면 cpu로 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device) # 모형을 GPU로 복사
model.train() # 학습모드로 전환

# 옵티마이저를 트랜스포머가 제공하는 AdamW로 설정
optim = AdamW(model.parameters(), lr=1e-5)

# 멀티클래스이므로 크로스 엔트로피를 손실 함수로 사용
criterion = torch.nn.CrossEntropyLoss()

start = time.time() # 시작시간 기록
num_epochs = 4  # 학습 epoch를 4회로 설정
for epoch in range(num_epochs):
    total_epoch_loss = 0 # epoch의 총 loss 초기화
    
    for step, batch in enumerate(train_loader):
        optim.zero_grad() # 그레이디언트 초기화
        
        # 배치에서 라벨을 제외한 입력한 추출해 GPU로 복사
        inputs= {k:v.to(device) for k,v in batch.items() if k!='labels'}
        
        labels = batch['labels'].to(device) # 배치에서 라벨을 추출해 GPU로 복사
        outputs = model(inputs) # 모형으로 결과 예측
        
        # 두 클래스에 대해 예측하고 각각 비교해야 하므로
        # labels에 대해 원핫 인코딩을 적용한 후에 손실을 계산
        loss = criterion(outputs, F.one_hot(labels, num_classes=2).float()) # loss 계산
        
        if (step + 1) % 100 ==0 : # 100 배치마다 경과한 시간과 loss를 출력
            elapsed = time.time() - start
            print(
                'Epoch %d, batch %d, elapsed time: % .2f, loss: %.4f'
                        % (epoch +1, step +1, elapsed, loss)
            )
            total_epoch_loss += loss
            loss.backward() # 그레이디언트 계산
            optim.step() # 가중치 계산
            
        avg_epoch_loss = total_epoch_loss / len(train_loader) # epoch의 평균 loss 계산
        print('Average loss for epoch %d: %.4f' % (epoch+1, avg_epoch_loss))            

In [ ]:
# Trainer를 이용한 미세조정학습의 성능인 87%보다도 나은 성능이다. 옵티마이저의 학습전략을 수정함으로써 성능의 향상이 가능하므로
# 트랜스포머가 제공하는 학습률 스케줄러를 적용한다면 좀 더 나은 성능을 얻을 수도 있을 것이다.

# 테스트 집합에 대해 성능 측정
from datasets import load_metric

test_loader = DataLoader(test_dataset, batch_size=16)

metric = load_metric('accuracy')
model.eval()
for batch in test_loader:
    inputs = {k: v.to(device) for k,v in batch.items() if k !='labels'}
    labels = batch['labels'].to(device)
    
    with torch.no_grad(): # 학습할 필요가 없으므로 그레이디언트 계산을 끔
        outputs = model(inputs)
        #print(outputs)
    predictions = torch.argmax(outputs, dim=1)
    metric.add_batch(predictions, references=labels)
    
metric.compute()        